In [1]:
from eda_imports import *

import matplotlib.patches as patches

### Load GTF

In [2]:
%time df_gtf = pd.read_pickle('/projects/btl2/zxue/gtf2csv/data/Homo_sapiens.GRCh37.75.pkl')

CPU times: user 1.38 s, sys: 915 ms, total: 2.3 s
Wall time: 2.31 s


In [4]:
ndf_gtf = df_gtf.query('feature != "gene"').query('feature != "transcript"').drop(['gene_source'], axis=1)

# See how to common it is to have everything the same but 3' UTR for two transcripts

In [5]:
ndf_gtf.head(2)

,seqname,source,feature,start,end,score,strand,frame,CCDS,ccds_id,cds_end_NF,cds_start_NF,exon_id,exon_number,gene_biotype,gene_id,gene_name,mRNA_end_NF,mRNA_start_NF,protein_id,seleno,transcript_id,transcript_name,transcript_source
2,1,processed_transcript,exon,11869,12227,.,+,.,NaN,NaN,NaN,NaN,ENSE00002234944,1.0,pseudogene,ENSG00000223972,DDX11L1,NaN,NaN,NaN,NaN,ENST00000456328,DDX11L1-002,havana
3,1,processed_transcript,exon,12613,12721,.,+,.,NaN,NaN,NaN,NaN,ENSE00003582793,2.0,pseudogene,ENSG00000223972,DDX11L1,NaN,NaN,NaN,NaN,ENST00000456328,DDX11L1-002,havana


In [6]:
ndf_gtf.source.value_counts().head()

protein_coding             1887971
nonsense_mediated_decay    279659 
processed_transcript       141165 
retained_intron            121455 
lincRNA                    36487  
Name: source, dtype: int64

In [7]:
ndf_gtf.feature.value_counts()

exon              1306656
CDS               791856 
UTR               304070 
stop_codon        73411  
start_codon       73358  
Selenocysteine    114    
Name: feature, dtype: int64

In [8]:
assert df_gtf.gene_id.unique().shape[0] == df_gtf[['gene_id', 'gene_biotype']].drop_duplicates().shape[0]

In [9]:
def count_transcripts_with_identical_cds(grp):
    """grp: grouped by gene_id"""
    cds_list = []
    for k, g in grp.query('source == "protein_coding"').groupby('transcript_id'):
        cds_start_end_pairs = g.query('feature == "CDS"')[['start', 'end']].sort_values(['start', 'end']).values.tolist()
        cds_start_end_pairs_str = str(cds_start_end_pairs)
        cds_list.append(cds_start_end_pairs_str)
    return pd.Series(cds_list).value_counts().max()

In [10]:
%%time
keys, args = [], []
grp_by_cols = ['gene_name', 'gene_id', 'gene_biotype']
# for k, grp in tqdm(ndf_gtf.groupby(grp_by_cols)): # raise exception for some reason: https://github.com/tqdm/tqdm/issues/323
for k, grp in ndf_gtf.groupby(grp_by_cols):
    keys.append(k)
    args.append(grp)

CPU times: user 14.7 s, sys: 730 ms, total: 15.4 s
Wall time: 15.4 s


In [11]:
%%time
with multiprocessing.Pool(50) as p:
    res = p.map(count_transcripts_with_identical_cds, args)

CPU times: user 18.7 s, sys: 4.05 s, total: 22.7 s
Wall time: 42.2 s


In [12]:
# number of genes with transcripts that share identical cds regions
df_count = pd.DataFrame(keys, columns=grp_by_cols)

In [13]:
df_count['num'] = res

In [15]:
df_count.head()

,gene_name,gene_id,gene_biotype,num
0,5S_rRNA,ENSG00000201285,rRNA,NaN
1,5S_rRNA,ENSG00000212595,rRNA,NaN
2,5S_rRNA,ENSG00000238602,rRNA,NaN
3,5S_rRNA,ENSG00000238762,rRNA,NaN
4,5S_rRNA,ENSG00000239156,rRNA,NaN


In [18]:
df_count.query('gene_biotype == "protein_coding"').shape

(22810, 4)

In [17]:
df_count.query('gene_biotype == "protein_coding"').query('num > 1').shape

(5888, 4)

In [19]:
5888 / 22810

0.2581323980710215

So 25% of the genes have protein_coding transcripts sharing identical CDS

In [20]:
ndf_count = df_count.query('gene_biotype == "protein_coding"')

In [23]:
ndf_count.shape

(22810, 4)

In [22]:
ndf_count.sort_values('num', ascending=False).head(10)

,gene_name,gene_id,gene_biotype,num
27249,MRPL55,ENSG00000162910,protein_coding,17.0
8341,BDNF,ENSG00000176697,protein_coding,14.0
6694,ANAPC11,ENSG00000141552,protein_coding,13.0
62119,ZBTB7C,ENSG00000184828,protein_coding,12.0
21573,IL32,ENSG00000008517,protein_coding,10.0
28934,NREP,ENSG00000134986,protein_coding,10.0
59633,UBA52,ENSG00000221983,protein_coding,10.0
7718,ARPP21,ENSG00000172995,protein_coding,10.0
5397,AGTR1,ENSG00000144891,protein_coding,9.0
16590,FAM107B,ENSG00000065809,protein_coding,9.0
